In [13]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

file_path = '../resource/data/data/'

In [2]:
import os
os.getcwd()

'C:\\Users\\USER\\Documents\\LGhv_practice\\ETL'

# 중복 데이터 처리

In [11]:
df= pd.DataFrame([['hi','hello','holla', 'hallo', 'hi'],
              ['ENG', 'USA', 'EPN', 'DEU', 'ENG']])

df = df.T #열과 행 전치

df.duplicated() #중복 확인

df.drop_duplicates(inplace = True)

df

,0,1
0,hi,ENG
1,hello,USA
2,holla,EPN
3,hallo,DEU


# 함수 적용

## apply

In [14]:
def f(data):
    return data+ '...'

#series에 함수 적용
df[0].apply(f) #Series는 셀 단위 적용 
df.apply(f) #DataFrame은 행이나 열 단위로 적용인데 기본은 열 단위 적용

df.applymap(f) #applymap은 셀 단위 적용

,0,1
0,hi...,ENG...
1,hello...,USA...
2,holla...,EPN...
3,hallo...,DEU...


In [18]:
import seaborn as sns
titanic = sns.load_dataset('titanic')

df = titanic[['age', 'fare']]
#df.head()

def min_max(data):
    return data.max() - data.min()
#행 단위로 함수 적용
result = df.apply(min_max, axis = 0)
result

age      79.5800
fare    512.3292
dtype: float64

## pipe

In [28]:
#하나의 데이터를 가지고 null여부를 판단해 주는 함수 대입
#cell 단위로 적용해서 DataFrame을 return
df.pipe(lambda x: x.isnull())

def missing_values(x):
    return x.isnull()

#집계를 하는 함수를 대입하면 열 단위로 대입해서 집계를 수행한 후 Series를 return
df.pipe(lambda x: missing_values(x)).sum()

def missing_counts(x):
    return missing_values(x).sum()

#Series를 return 하는 함수를 대입해서 하나의 값을 return
df.pipe(lambda x: missing_counts(x))

age     177
fare      0
dtype: int64

# 열 편집

## 열 분리

In [41]:
df = pd.read_excel(file_path+'주가데이터.xlsx')
#df.info()

#자료형이 datetime64라서 split 사용할 수 없음
#dates = df['연월일'].str.split('-')

#data 타입 변경
#astype 호출해서 변경
#map 함수에 형을 변경할 수 있는 함수를 대입

df['연월일'] = df['연월일'].astype('str')
dates = df['연월일'].str.split('-')
#dates

df['연'] = dates.str.get(0)
df['월'] = dates.str.get(1) 
df['일'] = dates.str.get(2)

df.head()

,연월일,당일종가,전일종가,시가,고가,저가,거래량,연,월,일
0,2018-07-02,10100,600,10850,10900,10000,137977,2018,07,02
1,2018-06-29,10700,300,10550,10900,9990,170253,2018,06,29
2,2018-06-28,10400,500,10900,10950,10150,155769,2018,06,28
3,2018-06-27,10900,100,10800,11050,10500,133548,2018,06,27
4,2018-06-26,10800,350,10900,11000,10700,63039,2018,06,26


# data 결합

## concat

In [47]:
df1 = pd.DataFrame({
    'a':['a0','a1','a2','a3'],
    'b':['b0','b1','b2','3b'],
    'c':['c0','c1','c2','c3']
}, index = [1,2,3,4])

df2 = pd.DataFrame({
    'a':['a2','a3','a4','a5'],
    'b':['b2','b3','b4','b5'],
    'c':['c2','c3','c4','c5']
}, index = [2,3,4,5])


pd.concat([df1, df2]) #기본적으로 행 방향으로 결합
#동일한 컬럼은 값이 있지만 한쪽에만 존재하는 컬럼은 반대 쪽에는 Nan이 됨

pd.concat([df1, df2],axis = 1) #동일한 인덱스 끼리 옆으로 결합
#한쪽에만 존재하는 데이터도 결합 - Outer Join

pd.concat([df1, df2], axis = 1, join = 'inner') #동일한 인덱스 끼리 옆으로 결합
#양쪽 모두에 존재하는 데이터만 결합 - Inner Join

,a,b,c,a,b,c
2,a1,b1,c1,a2,b2,c2
3,a2,b2,c2,a3,b3,c3
4,a3,3b,c3,a4,b4,c4


## append

In [48]:
df1.append(df2)

C:\Users\USER\AppData\Local\Temp\ipykernel_11816\3062608662.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(df2)


,a,b,c
1,a0,b0,c0
2,a1,b1,c1
3,a2,b2,c2
4,a3,3b,c3
2,a2,b2,c2
3,a3,b3,c3
4,a4,b4,c4
5,a5,b5,c5


## combine_first

In [61]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5,4.5,np.nan])
b = pd.Series(np.arange(len(a)), dtype=np.float64)

#조건이 True - data1, False - data2
#r = np.where(조건, 데이터1, 데이터2)

#a의 값이 NaN인 경우 - b의 값으로 채우기
r = np.where(pd.isnull(a),b, a)

a.combine_first(b), r #동일

(0    0.0
 1    2.5
 2    2.0
 3    3.5
 4    4.5
 5    5.0
 dtype: float64,
 array([0. , 2.5, 2. , 3.5, 4.5, 5. ]))

## merge

In [9]:
price = pd.read_excel(file_path+'stock_price.xlsx')
valuation = pd.read_excel(file_path+'stock_valuation.xlsx')

price.info(), valuation.info()

outer_join = pd.merge(price, valuation, how = 'outer')
inner_join = pd.merge(price, valuation)
#outer_join, inner_join
inner_join

,id,stock_name,value,price,name,eps,bps,per,pbr
0,130960,CJ E&M,58540.666667,98900,CJ E&M,6301.333333,54068,15.695091,1.829178
1,139480,이마트,239230.833333,254500,이마트,18268.166667,295780,13.931338,0.860437
2,145990,삼양사,82750.000000,82000,삼양사,5741.000000,108090,14.283226,0.758627
3,185750,종근당,40293.666667,100500,종근당,3990.333333,40684,25.185866,2.470259
4,204210,모두투어리츠,3093.333333,3475,모두투어리츠,85.166667,5335,40.802348,0.651359


## join

In [13]:
price = pd.read_excel(file_path+'stock_price.xlsx', index_col = 'id')
valuation = pd.read_excel(file_path+'stock_valuation.xlsx', index_col = 'id')

price.join(valuation) #instance를 method 호출
pd.DataFrame.join(price, valuation) #class로 instance method 호출 - unbound 호출

,stock_name,value,price,name,eps,bps,per,pbr
id,,,,,,,,
128940,한미약품,59385.666667,421000,NaN,NaN,NaN,NaN,NaN
130960,CJ E&M,58540.666667,98900,CJ E&M,6301.333333,54068.0,15.695091,1.829178
138250,엔에스쇼핑,14558.666667,13200,NaN,NaN,NaN,NaN,NaN
139480,이마트,239230.833333,254500,이마트,18268.166667,295780.0,13.931338,0.860437
142280,녹십자엠에스,468.833333,10200,NaN,NaN,NaN,NaN,NaN
145990,삼양사,82750.000000,82000,삼양사,5741.000000,108090.0,14.283226,0.758627
185750,종근당,40293.666667,100500,종근당,3990.333333,40684.0,25.185866,2.470259
192400,쿠쿠홀딩스,179204.666667,177500,NaN,NaN,NaN,NaN,NaN
199800,툴젠,-2514.333333,115400,NaN,NaN,NaN,NaN,NaN


# 그룹화

In [5]:
import seaborn as sns

In [6]:
titanic = sns.load_dataset('titanic')
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [7]:
#필요한 column 골라내기
df = titanic[['age','sex','class', 'fare', 'survived']]
df.head()

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0


In [8]:
grouped = df.groupby(['class'])
grouped

In [16]:
#key에는 그룹화 된 컬럼의 값이 전달되고 group에는 그룹화된 데이터가 전달됨
for key, group in grouped:
    print(key)

First
Second
Third


In [19]:
#그룹별 집계 가능
grouped.mean()

#개별 그룹 가져오기
group1 = grouped.get_group('First')
group1.head()

,age,sex,class,fare,survived
1,38.0,female,First,71.2833,1
3,35.0,female,First,53.1000,1
6,54.0,male,First,51.8625,0
11,58.0,female,First,26.5500,1
23,28.0,male,First,35.5000,1


In [21]:
grouped_two = df.groupby(['class','sex'])

for key, group in grouped_two:
    print(key)

('First', 'female')
('First', 'male')
('Second', 'female')
('Second', 'male')
('Third', 'female')
('Third', 'male')


In [22]:
print(type(df['class']))

<class 'pandas.core.series.Series'>


## 집계 함수 사용

In [24]:
print(grouped.std())
print(grouped.fare.std()) #.을 이용해서 컬럼에 접근 할 때는 컬럼 이름이 반드시 문자열

              age       fare  survived
class                                 
First   14.802856  78.380373  0.484026
Second  14.001077  13.417399  0.500623
Third   12.495398  11.778142  0.428949
class
First     78.380373
Second    13.417399
Third     11.778142
Name: fare, dtype: float64


## 사용자 정의 함수 

In [32]:
#사용자 정의 함수 적용

#group에 agg를  이용해서 적용하게 되면 매개 변수로 각 그룹이 대입되는 구조
def f(group):
    return (group.max(), group.min())

#하나의 함수 적용
agg_f = grouped.agg(f)
agg_f

#여러 개의 함수 적용 - list로 함수 넘겨 주기
agg_two = grouped.agg([min, max])
agg_two 

#컬럼 마다 다름 함수 적용 - dict로 함수 넘겨주기 
agg_three = grouped.agg({'fare': max, 'age': min})
agg_three

,fare,age
class,,
First,512.3292,0.92
Second,73.5000,0.67
Third,69.5500,0.42


In [38]:
#셀 단위로 함수 적용 - transform 이나 apply 사용

def z_score(x):
    return (x - x.mean()) / x.std()

age_zscore = grouped.age.transform(z_score)
age_zscore.head()

#SQL의 Having(Group By 이후의 조건 - 그룹화한 항목의 filtering)을 만들고자 할 때는 filter 함수 이용
#그룹화한 항목의 필터링은 그룹의 집계함수를 이용해서 필터링을 수행
#행의 개수 가 300개가 넘는 그룹의 데이터만 추출
group_filter = grouped.filter(lambda x: len(x) > 300)
group_filter.head()

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
2,26.0,female,Third,7.9250,1
4,35.0,male,Third,8.0500,0
5,NaN,male,Third,8.4583,0
7,2.0,male,Third,21.0750,0


## multi - indexing

In [41]:
#2개의 컬럼으로 그룹화 - 멀티 인덱스 생성
grouped = df.groupby(['class', 'sex'])

#First class이고 sex가 male인 데이터만 골라오기
gdf = grouped.mean()
gdf.loc[('First', 'male')]
gdf.xs('male', level = 'sex')

,age,fare,survived
class,,,
First,41.281386,67.226127,0.368852
Second,30.740707,19.741782,0.157407
Third,26.507589,12.661633,0.135447


## pivot table

In [46]:
pd.pivot_table(df, index = 'class', columns = 'sex', values = 'age', aggfunc = 'mean')

sex,female,male
class,,
First,34.611765,41.281386
Second,28.722973,30.740707
Third,21.750000,26.507589


# data 구조화

In [48]:
#multi index로 사용할 데이터 생성
mul_index = pd.MultiIndex.from_tuples([('clust_1', '2015'), ('clust_1', '2016'),
                                      ('clust_2', '2015'), ('clust_2', '2015')])
data = pd.DataFrame(data = np.arange(16).reshape(4,4),
                   index = mul_index, columns = ['prd_1', 'prd_2', 'prd_3', 'prd_4'],
                   dtype = 'int')

data

prd_1  prd_2  prd_3  prd_4
clust_1 2015      0      1      2      3
        2016      4      5      6      7
clust_2 2015      8      9     10     11
        2015     12     13     14     15

## stack

In [49]:
data_stacked = data.stack(dropna = False)
data_stacked

clust_1  2015  prd_1     0
               prd_2     1
               prd_3     2
               prd_4     3
         2016  prd_1     4
               prd_2     5
               prd_3     6
               prd_4     7
clust_2  2015  prd_1     8
               prd_2     9
               prd_3    10
               prd_4    11
               prd_1    12
               prd_2    13
               prd_3    14
               prd_4    15
dtype: int32

In [ ]:
data.loc[]

## unstack

In [51]:
#data_stacked.unstack(level = 0) #0레벨의 인덱스가 column으로 변경됨

## 시계열 데이터 재 구조화

In [52]:
data = pd.read_csv(file_path+'/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [54]:
#시계열 인덱스 생성
periods = pd.PeriodIndex(year = data['year'], quarter= data['quarter'], name = 'date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [55]:
columns = pd.Index(['realgdp', 'infi', 'unemp'], name = 'item')
data = data.reindex(columns = columns)
data.head()

item,realgdp,infi,unemp
0,2710.349,NaN,5.8
1,2778.801,NaN,5.1
2,2775.488,NaN,5.3
3,2785.204,NaN,5.6
4,2847.699,NaN,5.2


In [56]:
#인덱스에 날짜 설정
data.index = periods.to_timestamp('D', 'end') #쿼터의 마지막 날을 세부 데이터로 설정
data

item,realgdp,infi,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,NaN,5.8
1959-06-30 23:59:59.999999999,2778.801,NaN,5.1
1959-09-30 23:59:59.999999999,2775.488,NaN,5.3
1959-12-31 23:59:59.999999999,2785.204,NaN,5.6
1960-03-31 23:59:59.999999999,2847.699,NaN,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,13324.600,NaN,6.0
2008-12-31 23:59:59.999999999,13141.920,NaN,6.9
2009-03-31 23:59:59.999999999,12925.410,NaN,8.1


In [57]:
#날짜를 데이터로 사용
ldata = data.stack().reset_index().rename(columns = {0:'value'})
ldata

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,unemp,5.800
2,1959-06-30 23:59:59.999999999,realgdp,2778.801
3,1959-06-30 23:59:59.999999999,unemp,5.100
4,1959-09-30 23:59:59.999999999,realgdp,2775.488
...,...,...,...
401,2009-03-31 23:59:59.999999999,unemp,8.100
402,2009-06-30 23:59:59.999999999,realgdp,12901.504
403,2009-06-30 23:59:59.999999999,unemp,9.200
404,2009-09-30 23:59:59.999999999,realgdp,12990.341


In [58]:
pivoted = ldata.pivot('date','item','value')
pivoted

item,realgdp,unemp
date,,
1959-03-31 23:59:59.999999999,2710.349,5.8
1959-06-30 23:59:59.999999999,2778.801,5.1
1959-09-30 23:59:59.999999999,2775.488,5.3
1959-12-31 23:59:59.999999999,2785.204,5.6
1960-03-31 23:59:59.999999999,2847.699,5.2
...,...,...
2008-09-30 23:59:59.999999999,13324.600,6.0
2008-12-31 23:59:59.999999999,13141.920,6.9
2009-03-31 23:59:59.999999999,12925.410,8.1


## melt

In [59]:
data = pd.DataFrame({'clust_id':[1,2,3,4], 'prod_id':['A','B','C','D'],
                     'pch_cnt':[1,2,3,4], 'pch_price': [100,200,300,400],
                     'pch_amt':[10,20,30,40]
})
result = pd.melt(data, id_vars = ['clust_id', 'prod_id'])
result

,clust_id,prod_id,variable,value
0,1,A,pch_cnt,1
1,2,B,pch_cnt,2
2,3,C,pch_cnt,3
3,4,D,pch_cnt,4
4,1,A,pch_price,100
5,2,B,pch_price,200
6,3,C,pch_price,300
7,4,D,pch_price,400
8,1,A,pch_amt,10
9,2,B,pch_amt,20


## cross tabs

In [63]:
data = pd.DataFrame({'id':['id1', 'id1', 'id1', 'id2', 'id2', 'id3'], 
                     'fac_1':['a','a','a','b','b','b'],
                     'fac_2':['d','d','d','c','c','d'], 
})
#fac_1과 fac_2의 빈도 분석
pd.crosstab(data['fac_1'], data['fac_2'],
           rownames = ['사실'], colnames = ['거짓'], normalize = True)
pd.crosstab(data.id, [data['fac_1'], data['fac_2']])


fac_1  a  b   
fac_2  d  c  d
id            
id1    3  0  0
id2    0  2  0
id3    0  0  1